In [ ]:
import torch
import numpy as np

import os, sys
project_root_dir = os.path.join(os.getcwd(),'../..')
if project_root_dir not in sys.path:
    sys.path.append(project_root_dir)
    import config

In [ ]:
from FeatureSelection.Experiments.doc import ExperimentDoc

def generate_figures(exp) -> None:
    save_dir = os.path.join(exp.config()['save_result_dir'], 'imgs')
    
    # Histrogram
    hist_fig = ExperimentDoc.hist_phi(exp)
    hist_fig.savefig(os.path.join(save_dir, 'hist_phi.pdf'), bbox_inches='tight')

    # Band Selected
    dataset = exp.dataset
    X, y = dataset[:]

    # get 1 samper per class
    samples = np.zeros((3, 128))
    sample_labels = ['Healthy', 'Tumor', 'Hypervascular']
    for i in range(0, 3):
        idx = np.where(y == i)[0][0]
        samples[i] = X[idx].numpy()

    samples = dataset.transform.inverse_transform(torch.tensor(samples))
    wv = np.round(dataset.wv, 0).astype(int)

    selected_band_fig = ExperimentDoc.plot_band_selection(exp, samples=samples, labels=sample_labels, wv=wv)
    selected_band_fig.savefig(os.path.join(save_dir, 'selected_band.pdf'), bbox_inches='tight')


In [ ]:
from FeatureSelection.Experiments.BrainCancerHSIBandSelection import VNIRimagesOp8C1
experiments = [VNIRimagesOp8C1]

for _exp in experiments:
    exp = _exp()
    exp.run()
    exp.save_config()
    exp.save_results()
    generate_figures(exp)

In [ ]:
from FeatureSelection.Experiments.BrainCancerHSIBandSelection import VNIRimagesOp12C1
experiments = [VNIRimagesOp12C1]

for _exp in experiments:
    exp = _exp()
    exp.run()
    exp.save_config()
    exp.save_results()
    generate_figures(exp)

In [ ]:
from FeatureSelection.Experiments.BrainCancerHSIBandSelection import VNIRimagesOp15C1, VNIRimagesOp20C1, VNIRimagesOp12C2, VNIRimagesOp8C2
experiments = [VNIRimagesOp15C1, VNIRimagesOp20C1, VNIRimagesOp12C2, VNIRimagesOp8C2]

for _exp in experiments:
    exp = _exp()
    print(exp.experiment)
    exp.run()
    exp.save_config()
    exp.save_results()
    generate_figures(exp)

In [ ]:
from FeatureSelection.Experiments.BrainCancerHSIBandSelection import VNIRimagesOpAll_KMeans, VNIRimagesOpAll_OCSP
experiments = [VNIRimagesOpAll_KMeans, VNIRimagesOpAll_OCSP]

for _exp in experiments:
    exp = _exp()
    print(exp.experiment)
    exp.run()
    exp.save_config()
    exp.save_results()
    generate_figures(exp)

# Laparoscopy

In [ ]:
import torch
import numpy as np

import os, sys
project_root_dir = os.path.join(os.getcwd(),'../..')
if project_root_dir not in sys.path:
    sys.path.append(project_root_dir)
    import config

In [ ]:
from FeatureSelection.Experiments.LaparoscopyHSIBandSelection import ExperimentFactory, ExperimentType
from FeatureSelection.Experiments.doc import ExperimentDoc

def generate_figures(exp) -> None:
    save_dir = os.path.join(exp.config()['save_result_dir'], 'imgs')
    
    # Histrogram
    hist_fig = ExperimentDoc.hist_phi(exp)
    hist_fig.savefig(os.path.join(save_dir, 'hist_phi.pdf'), bbox_inches='tight')

    # Band Selected
    dataset = exp.dataset
    X, y = dataset[:]

    # get 1 samper per class
    samples = np.zeros((4, 68))
    sample_labels = ['Fat', 'Muscle', 'Nerve', 'Vessel']
    for i in range(0, 4):
        idx = np.where(y == i)[0][0]
        samples[i] = X[idx].numpy()

    samples = dataset.transform.inverse_transform(torch.tensor(samples))
    # wv = np.round(dataset.wv, 0).astype(int)
    wv = np.array(exp.dataset.wv).astype(int)

    selected_band_fig = ExperimentDoc.plot_band_selection(exp, samples=samples, labels=sample_labels, wv=wv)
    selected_band_fig.savefig(os.path.join(save_dir, 'selected_band.pdf'), bbox_inches='tight')


In [ ]:
# exp_type = [ExperimentType.Concrete_2, ExperimentType.Concrete_3, ExperimentType.Concrete_5, ExperimentType.Gaussian_5]
# reg_factors = [1, 1.5, 2, 3]

# for _exp_type in exp_type:
#     for _reg_factor in reg_factors:
#         factory = ExperimentFactory()
#         exp = factory.create_experiment(_exp_type, reg_factor=_reg_factor)
#         print(exp.experiment)
#         exp.run()
#         exp.save_config()
#         exp.save_results()
#         generate_figures(exp)


In [ ]:
import torch
import numpy as np

import os, sys
project_root_dir = os.path.join(os.getcwd(),'../..')
if project_root_dir not in sys.path:
    sys.path.append(project_root_dir)
    import config

In [ ]:
from FeatureSelection.Experiments.LaparoscopyHSIBandSelection import ExperimentFactory, ExperimentType
factory = ExperimentFactory()
exp = factory.create_experiment(ExperimentType.Concrete_IP)

In [ ]:
exp.model.feature_selector.matrix_estimator

In [ ]:
exp.run()

In [ ]:
exp.save_config()
exp.save_results()
generate_figures(exp)

In [ ]:
exp.ip.to_df()

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colormaps as cm
def moving_average(a, n=10, padding_size=0) :
    # ret = np.cumsum(a, dtype=float) if padding_size == 0 else np.cumsum(np.insert(a, 0, np.zeros(padding_size)), dtype=float)
    ret = np.cumsum(a, dtype=float) if padding_size == 0 else np.cumsum(np.insert(a, 0, np.ones(padding_size)*a[0]), dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def show_ip(ip_df: pd.DataFrame, n=25, labels=[], moving_average_n=25):
    '''
        Create a Information Plane illustration

        Parameters
        ----------
        ip_df : pd.Dataframe, dataframe which contains the MI. This dataframe
            has a specific structure which is created from IPDL.InformationPlane class
        n: int, Number of samples to visualize, sampling generated by a log-scale
    '''
    colors = ['Greys', 'Reds', 'Blues', 'Greens', 'Oranges',
                'Purples', 'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd',
                'RdPu', 'BuPu', 'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn',
                'BuGn', 'YlGn']

    with plt.style.context('seaborn'):
        fig = plt.figure(figsize=(6, 4),constrained_layout=False)
        gs1 = fig.add_gridspec(nrows=1, ncols=1, left=0.08, right=0.84, wspace=0.05)
        gs2 = fig.add_gridspec(nrows=1, ncols=len(ip_df.keys()[::2]), left=0.85, right=0.95, wspace=0)
        f8_ax1 = fig.add_subplot(gs1[:, :])
        f8_ax1.set_xlabel("$\mathcal{I}(X; T)$", fontsize=14)
        f8_ax1.set_ylabel("$\mathcal{I}(T; Y)$", fontsize=14)

        for idx, (layer, _) in enumerate(ip_df.columns[::2]):
            Ity = moving_average(ip_df[layer]['Ity'].to_numpy(), n=moving_average_n, padding_size=int(moving_average_n*0.75))
            Ixt = moving_average(ip_df[layer]['Ixt'].to_numpy(), n=moving_average_n, padding_size=int(moving_average_n*0.75))
            
            cmap = cm[colors[idx]]
            iterations = np.geomspace(1, len(Ity)-1, num=n, dtype=np.uint)

            color = np.array([cmap(iterations[int(len(iterations)*0.95)])])
            sc = f8_ax1.scatter(Ixt[iterations], Ity[iterations], c=iterations, vmin=0, vmax=iterations.max(), cmap=cmap, edgecolor=color)
            if not labels:
                f8_ax1.scatter([], [], c=color, label=layer)
            else:
                f8_ax1.scatter([], [], c=color, label=labels[idx])

            f8_ax2 = fig.add_subplot(gs2[0, idx])
            cb = fig.colorbar(sc, cax=f8_ax2, pad=0)
            cb.set_ticks([])

        f8_ax1.legend()
        cb.set_ticks([0, iterations.max()])
        f8_ax2.set_yticklabels(['0', ip_df[layer]['Ixt'].size - 1])
        cb.set_label("Iterations", labelpad=-18)

    return fig

In [ ]:
ip_df = exp.ip.to_df()
fig = show_ip(ip_df, 150, moving_average_n=20, labels=['$X_s$', '$L_1$', '$L_2$', '$L_3$'])
save_dir = os.path.join(exp.config()['save_result_dir'], 'imgs')
fig.savefig(os.path.join(save_dir, 'ip.pdf'), bbox_inches='tight')